# Дембицкий М.Е. Домашнее задание № 1

## Оглавление
[Задание 1. Реализовать метрики Recall@k и Money Recall@k](#task1)  
[Задание 2. Реализовать метрику MRR@k](#task2)  
[Задание 3*. Реализовать метрику nDCG@k](#task3)  

In [21]:
import numpy as np
import pandas as pd
import math

In [7]:
# data
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
prices_recommended_list=[180,40,50,800,20,550,160,120,110,330]

bought_list = [521, 32, 143, 991]
prices_bought=[480,30,180,800]

<a name="task1"></a>
## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [8]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall #Добавьте сюда результат расчета метрики


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    #TODO: Ваш код здесь
    flags = np.isin(recommended_list, bought_list)
    recall  = np.dot(flags, prices_recommended)/prices_bought.sum()
    return recall #Добавьте сюда результат расчета метрики

In [31]:
print(f'Ответ: Recall@k = {recall_at_k(recommended_list, bought_list)}')
print(f'Ответ: Money Recall@k = {money_recall_at_k(recommended_list, bought_list, prices_recommended_list, prices_bought)}')

Ответ: Recall@k = 0.5
Ответ: Money Recall@k = 0.6577181208053692


<a name="task2"></a>
## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [15]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    amount_user = len(bought_list)
    rr = []
    for i in np.arange(amount_user):    
        relevant_indexes = np.nonzero(np.isin(recommended_list[i][:k], bought_list[i]))[0]
        if len(relevant_indexes) != 0:
            rr.append(1/(relevant_indexes[0]+1))
    
    if len(rr) == 0:
        return 0
    mrr  = sum(rr)/amount_user
    return  mrr #Добавьте сюда результат расчета метрики

In [13]:
recommended_list_3_users = [[143,156,1134,991,27,1543,3345,533,11,43],
                            [1134,533,14,4,15,1543,1,99,27,3345],
                            [991,3345,27,533,43,143,1543,156,1134,11]]

bought_list_3_users= [[521,32,143], #user1
                      [143,156,991,43,11], #user2
                      [1,2]] #user3

In [30]:
print(f'Ответ: MRR@k = {mrr_at_k(recommended_list_3_users, bought_list_3_users)}')

Ответ: MRR@k =0.3333333333333333


<ipython-input-15-7e9ed913177d>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bought_list = np.array(bought_list)


<a name="task3"></a>
## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [28]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    recommended_list = np.array(recommended_list)[:k]

    
    flags = np.isin(recommended_list, bought_list)
    rank_list = []
    for i in np.arange(len(recommended_list)):
        if i < 2:
            rank_list.append(i+1)
        else:
            rank_list.append(math.log2(i+1))
    if len(recommended_list) == 0:
        return 0
    dcg = sum(np.divide(flags, rank_list)) / len(recommended_list)

    i_dcg = sum(np.divide(1, rank_list)) / len(recommended_list)
#     print(i_dcg)
    ndcg = dcg/i_dcg
    return ndcg #Добавьте сюда результат расчета метрики
    

In [32]:
print(f'Ответ: nDCG@k ={ndcg_at_k(recommended_list, bought_list)}')

Ответ: nDCG@k =0.489938890671454
